<a href="https://colab.research.google.com/github/cedro3/HR-VITON/blob/main/HR_VITON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #**セットアップ**

# githubからコードをコピー
! git clone https://github.com/cedro3/HR-VITON.git
%cd HR-VITON

# ライブラリーのインストール
! pip install tensorboardX
! pip install torchgeometry

# 学習済みパラメータのダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1XJTCdRBOPVgVTmqzhVGFAgMm2NLkw5uQ', 'mtviton_step_100000.pth', quiet=False)
gdown.download('https://drive.google.com/uc?id=1BkSA8UJo-6eOkKcXTFOHK80Esc4vBmVC', 'gen_step_110000.pth', quiet=False)

# テストデータのダウンロード
gdown.download('https://drive.google.com/uc?id=1CcgCubhLc9iF6jGACdUgGhTDWMC7Gjzr', 'test.zip', quiet=False)
! unzip test.zip

# 関数のインポート
from function import *

# warning非表示
import warnings
warnings.filterwarnings("ignore")

# ダウンロードフォルダ作成
! mkdir download

In [ ]:
#@title #**人と衣服のランダム選択**
import glob
import random
import shutil
import os

# 乱数シード設定
seed_number =  120#@param {type:"integer"}
random.seed(seed_number)

# imageのランダム選択
reset_folder('image')
image_files = sorted(glob.glob('test/test/image/*.jpg'))
cnt = len(image_files)
num = random.sample(range(cnt),5)
image_names = []

for i in num:
  shutil.copy(image_files[i], 'image/'+os.path.basename(image_files[i]))
  image_names.append(image_files[i])
image_names.sort()
display_pic('image')


# clothのランダム選択
reset_folder('cloth')
cloth_files = sorted(glob.glob('test/test/cloth/*.jpg'))
cnt = len(cloth_files)
num = random.sample(range(cnt),5)
cloth_names =[]

for j in num:
  shutil.copy(cloth_files[j], 'cloth/'+os.path.basename(cloth_files[j]))
  cloth_names.append(cloth_files[j])
cloth_names.sort()
display_pic('cloth')


# txt作成
if os.path.isfile('test/test/test_pairs.txt'):
  os.remove('test/test/test_pairs.txt')

f = open('test/test/test_pairs.txt', 'w', encoding='UTF-8')
for image_name in image_names:
    for cloth_name in cloth_names:
        f.write(os.path.basename(image_name)+' ')
        f.write(os.path.basename(cloth_name)+'\n')
f.close()

In [ ]:
#@title #**仮想試着の実行**
reset_folder('output')

! python3 test_generator.py --occlusion --tocg_checkpoint './mtviton_step_100000.pth'\
                             --gpu_ids 0\
                             --gen_checkpoint './gen_step_110000.pth'\
                             --datasetting unpaired\
                             --dataroot './test'\
                             --data_list './test/test_pairs.txt'

clear_output()
display_pic_png('output/mtviton_step_100000.pth/test/unpaired/generator/output')

In [ ]:
#@title #**動画の作成**

import cv2
import glob

reset_folder('movie1')
reset_folder('movie2')

result_files = sorted(glob.glob('output/mtviton_step_100000.pth/test/unpaired/generator/output/*.png'))
cnt = 0
black = cv2.imread('black.jpg')
for image_name in image_names:
   for cloth_name in cloth_names:
      left = cv2.imread(image_name)
      center = cv2.imread(cloth_name)
      right = cv2.imread(result_files[cnt])
      tmp = cv2.hconcat([left, center])
      img1 = cv2.hconcat([tmp, right])
      cv2.imwrite('movie1/'+str(cnt).zfill(4)+'.jpg', img1)

      up = cv2.hconcat([black, center])
      down = cv2.hconcat([left, right])
      img2 = cv2.vconcat([up, down])
      cv2.imwrite('movie2/'+str(cnt).zfill(4)+'.jpg', img2)
      cnt +=1

! ffmpeg -y -r 1 -i movie1/%04d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error output1.mp4
! ffmpeg -y -r 1 -i movie2/%04d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error output2.mp4
display_mp4('output1.mp4')

In [ ]:
#@title #**動画のダウンロード**

from google.colab import files

square = True #@param {type:"boolean"}
if square == True:
  shutil.copy('output2.mp4', 'download/'+str(seed_number)+'_s.mp4')
  files.download('download/'+str(seed_number)+'_s.mp4')
else:
  shutil.copy('output1.mp4', 'download/'+str(seed_number)+'.mp4')
  files.download('download/'+str(seed_number)+'.mp4')
